# Model Debug 

## 1. Import all needed packages

In [3]:
import sys
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pylab as plt
from lmfit import minimize, Parameters, report_fit
from numpy import exp, log, poly1d, pi
import os

## 2. Define Residual Functions

In [4]:
def resi(fit, t, data):
    ''' Function to compute resiudals of fitted model using OLS method'''
    # Construct the fitted polynomial equation
    my_fit = poly1d(fit)
    #Compute predicted values
    ypred = my_fit(t)
    # Calculating residuals
    return ypred - data

def resi_bri(params, t, data):
    ''' create a briere model and subtract data '''
    # Get an ordered dictionary of parameter values
    v = params.valuesdict()
    # Briere Model
    model = v['B0'] * t * (t - v['T0']) * (abs(v['Tm'] - t) ** (1/2)) * \
        ((t < v['Tm']).astype(float)) * ((t > v['T0']).astype(float))
    return model - data


def resi_sch_log(params, T, log_data):
    ''' create a SchoolField model and subtract data '''
    k = 8.617e-5
    # Get an ordered dictionary of parameter values
    v = params.valuesdict()
    B0 = v['B0']
    E = v['E']
    Th = v['Th']
    Eh = v['Eh']
    model = log(B0) - E/k * (1/T - 1/283.15) - log(1 + exp(Eh/k * (1/Th - 1/T)))
    return model - log_data

def resi_sch_orig(params, T, data):
    ''' create a SchoolField model and subtract data '''
    k = 8.617e-5
    # Get an ordered dictionary of parameter values
    v = params.valuesdict()
    B0 = v['B0']
    E = v['E']
    Th = v['Th']
    Eh = v['Eh']
    # SchoolField Model
    # model = log(B0) - E * (T + 1/(283.15 * k)) - log(1 + exp(Eh * (1/(k*Th) - T)))
    model = log(B0) - E/k * (1/T - 1/283.15) - log(1 + exp(Eh/k * (1/Th - 1/T)))
    return exp(model) - data

## 3. Define Tool Functions

In [5]:
def get_AIC(residuals, data, params):
    n = len(data)
    RSS = sum(residuals ** 2)
    p_model = len(params)
    return n + 2 + n * log((2 * pi) / n) + n * log(RSS) + 2 * p_model

def get_BIC(residuals, data, params):
    n = len(data)
    RSS = sum(residuals ** 2)
    p_model = len(params)
    return n + 2 + n * log((2 * pi) / n) + n * log(RSS) + p_model * log(n)

def fitting_plot_OLS(x, fit_model=None, residuals_model=None, name=None, color=None):
    ''' Function to plot OLS fitted model '''
    for elem in range(len(fit_model)):
        plt.rcParams['figure.figsize'] = [20, 15]
        result_linear = np.poly1d(fit_model[elem])(x)
        plt.plot(x, result_linear, '.', markerfacecolor=color[elem],
                 markeredgecolor=color[elem], markersize=15, label=name[elem])
        # Get a smooth curve by plugging a time vector to the fitted linear model
        t_vec = np.linspace(min(x), max(x), 1000)
        ypred_smooth = np.poly1d(fit_model[elem])(t_vec)
        plt.plot(t_vec, ypred_smooth, color[elem], linestyle='--', linewidth=2)

def models_visualisation(id, d, params):
    """ To visualise 4 models """
    sub = d[d["ID"] == id]
    T = np.asarray(sub["ConTemp"])  # Temperature in kelvin
    T_kelvin = np.asarray(sub["ConTemp_Kelvin"])
    # Unlogged Trait value
    B = np.asarray(sub["OriginalTraitValue"])
    B_name = sub["OrignalTraitName"].iloc[0]
    B_unit = sub["OriginalTraitUnit"].iloc[0]

    fitting_plot_OLS(
        x=T,
        fit_model=(params[0], params[1]),
        name=("Quadratic", "Cubic"),
        color=("red", "Orange")
    )
    if params[2] != 0:
        # Briere model
        result_model = B + resi_bri(params[2], T, B)
        plt.plot(T, result_model, '.', markerfacecolor="blue",
                markeredgecolor="blue", markersize=15, label="Briere")
        # Get a smooth curve by plugging a time vector to the fitted non-linear model
        t_vec = np.linspace(min(T), max(T), 1000)
        N_vec = np.ones(len(t_vec))
        residual_smooth = resi_bri(params[2], t_vec, N_vec)
        plt.plot(t_vec, residual_smooth + N_vec,
                "blue", linestyle='--', linewidth=2)

    if params[3] != 0 and params[3] != None:
        # Simplified Schoolfield
        result_model = B + resi_sch_orig(params[3], T_kelvin, B)
        plt.plot(T, result_model, '.', markerfacecolor="magenta",
                markeredgecolor="magenta", markersize=15, label="Schoolfield")
        # Get a smooth curve by plugging a time vector to the fitted non-linear model
        t_vec = np.linspace(min(T_kelvin), max(T_kelvin), 1000)
        N_vec = np.ones(len(t_vec))
        residual_smooth = resi_sch_orig(params[3], t_vec, N_vec)
        plt.plot(t_vec - 273.15, residual_smooth + N_vec,
                "magenta", linestyle='--', linewidth=2)
        
    # Plot data points
    plt.plot(T, B, 'r+', markersize=15,
             markeredgewidth=2, label='Data')
    # Plot legend
    plt.legend(fontsize=20)
    plt.xlabel('Temperature(°C)', fontsize=20)
    plt.ylabel('%s\n%s' % (B_name, B_unit), fontsize=20)
    plt.ticklabel_format(style='scientific', scilimits=[0, 3])

## 4. Define Model Functions

### (1) Briere Model Function

In [6]:
def Briere_model(id, data, min_round, max_round, only_return_info = True):
    """ Briere model fitting  """
    #Variables
    T = np.asarray(data['ConTemp'][data['ID'] == id])  # Temperature
    B = np.asarray(data['OriginalTraitValue'][data['ID'] == id])  # Trait Value
    
    # Starting values
    B0 = 0.01
    T0 = min(T)
    Tm = max(T)

    # Initiate an empty dictionary for results
    results_all = {"bri.id": [id],
                   "bri.B0": [B0],
                   "bri.T0": [T0],
                   "bri.chisqr": [np.NAN],
                   "bri.Rsq": [np.NAN],
                   "bri.aic": [np.NAN],
                   "bri.bic": [np.NAN],
                   "bri.aicc": [np.NAN]
                   }

    best_p = 0
    round = 0
    while round < max_round:
        round += 1
        # If TPC converged within mean rounds, or max round is exceeded, break the loop
        if results_all['bri.aicc'] != [np.NaN] and round > min_round and results_all["bri.Rsq"] >= -1:
            break

        #Starting values defined for rounds
        #For the first round use setted starting values
        if round == 1:
            p = Parameters()
            p.add_many(("B0", B0), ("T0", T0), ("Tm", Tm))

        #For other rounds, assign B0 with random number distributed between 0 and B0 * 2
        else:
            p = Parameters()
            p.add("B0", value=np.random.normal(B0, 10))
            p.add("T0", value=np.random.normal(T0, 20), min=-100, max=60)
            p.add("Tm", value=np.random.normal(Tm, 40), min=-50, max=140)

        try:
            output = minimize(resi_bri, p, args=(T, B))
            pm = output.params
            rss = sum(resi_bri(pm, T, B) ** 2)
            tss = sum((B - np.mean(B)) ** 2)
            rsq = 1 - (rss/tss)
            aic = get_AIC(resi_bri(pm, T, B), B, pm)
            bic = get_BIC(resi_bri(pm, T, B), B, pm)
            aicc = aic + 2 * 3 * ((3 + 1)/(len(T) - 3 - 1))

            # Replace the fitting results if aicc is smaller,
            # or if the previous fit didn't converge
            if aic < results_all["bri.aic"] or results_all["bri.aic"] == [np.NaN] and rsq >= -1:
                best_p = output.params
                results_all = {
                    "bri.id": [id],
                    "bri.B0": [output.params["B0"].value],
                    "bri.T0": [output.params["T0"].value],
                    "bri.Tm": [output.params["Tm"].value],
                    "bri.chisqr": [output.chisqr],
                    "bri.Rsq": rsq,
                    "bri.aic": [aic],
                    "bri.bic": [bic],
                    "bri.aicc": [aicc]
                }
        except ValueError:
            pass
        continue
    
    if only_return_info == True:
        return pd.DataFrame(results_all)
    else:
        return pd.DataFrame(results_all), best_p

### (2) Define Schoolfield Model

In [7]:
# Schoolfield model simplified, appropriate for without low termperature data
def Schoolfield_model(id, data, min_round, max_round, only_return_info = True):
    """ fitting simplified Schoolfield model with some rounds """
    #Variables
    # k = 8.617e-5
    sub = data.loc[data["ID"] == id]

    T = np.asarray(sub["ConTemp_Kelvin"])  # Temperature in kelvin
    Th_min = np.min(T)
    Th_max = np.max(T)

    # kT = np.asarray(sub["Transferred_kT"]) # (1/k*T) in Kelvin
    B = np.asarray(sub["OriginalTraitValue"])  # Unlogged Trait value
    B0_min = np.min(B)
    B0_max = np.max(B)

    B_log = np.asarray(sub["LoggedOriginalTraitValue"])  # Logged Trait value
    
    ArrInf_bef, ArrInfo_af = Arrhenius_model(id, data)

    ArrParam_bef = ArrInf_bef[["B0_log_before_deactivation", "E_before_deactivation"]]
    ArrParam_af = ArrInfo_af[["B0_log_after_deactivation", "E_after_deactivation"]]
    
    T_max_C = sub["ConTemp_Kelvin"][sub["LoggedOriginalTraitValue"] == np.max(sub["LoggedOriginalTraitValue"])]
    
    if len(T_max_C) >= 2:
        T_max_C = np.min(T_max_C)
    
    # Starting values(matters on converge)
    B0 = float((ArrParam_bef["B0_log_before_deactivation"]))
    E = float(ArrParam_bef["E_before_deactivation"])
    Th = float(T_max_C)
    Eh = float(ArrParam_af["E_after_deactivation"])

    # Add starting values with bounding
    p = Parameters()
    p.add("B0", value=B0, min = B0_min, max = B0_max)
    p.add("E", value=E, min=0, max=4)
    p.add("Th", value=Th, min = Th_min, max = Th_max)
    p.add("Eh", value=Eh, min=0.01, max = 6)

    # Initiate an empty dictionary for results
    results_all = {
        "sch.id": [id],
        "sch.B0": [B0],
        "sch.E": [E],
        "sch.Th": [Th],
        "sch.Eh": [Eh],
        "sch.chisqr": [np.NAN],
        "sch.Rsq": [np.NAN],
        "sch.aic": [np.NAN],
        "sch.bic": [np.NAN],
        "sch.aicc": [np.NAN],
        "sch.Rsq_log": [np.NAN],
        "sch.aic_log": [np.NAN],
        "sch.bic_log": [np.NAN],
        "sch.aicc_log": [np.NAN]
    }
    
    best_p = 0
    round = 0

    while round < max_round:

        round += 1
        if results_all["sch.aicc"] != [np.NaN] and round > min_round and results_all['sch.Rsq'] >= -1:
            break

        #Starting values defined for rounds
        #For the first round use set starting values
        if round == 1:
            p = Parameters()
            p.add("B0", value=B0, min=B0_min, max=B0_max)
            p.add("E", value=E, min=0, max=4)
            p.add("Th", value=Th, min = Th_min, max = Th_max)
            p.add("Eh", value=Eh, min=0, max = 6)
        else:
            p = Parameters()
            p.add("B0", value=np.random.normal(loc=B0, scale=45), min = B0_min, max = B0_max)
            p.add("E", value=np.random.normal(loc=E, scale=3), min=0, max=3)
            p.add("Th", value=np.random.normal(Th, 30), min=Th_min, max=Th_max)
            p.add("Eh", value=np.random.normal(loc = Eh, scale=4), min=0, max = 6)

        #Results for model fitting on logged values
        try:
            output = minimize(resi_sch_log, p, args=(T, B_log))
            rss_log = sum(resi_sch_log(output.params, T, B_log) ** 2)
            tss_log = sum((B_log - np.mean(B_log)) ** 2)
            rsq_log = 1 - (rss_log/tss_log)
            aic_log = get_AIC(resi_sch_log(
                output.params, T, B_log), B_log, output.params)
            bic_log = get_BIC(resi_sch_log(
                output.params, T, B_log), B_log, output.params)
            aicc_log = aic_log + 2 * 4 * ((4 + 1)/(len(T) - 4 - 1))

            # Calculate unlogged fitting
            rss = sum(resi_sch_orig(output.params, T, B) ** 2)
            tss = sum((B - np.mean(B)) ** 2)
            rsq = 1 - (rss / tss)
            aic = get_AIC(resi_sch_orig(output.params, T, B), B, output.params)
            bic = get_BIC(resi_sch_orig(output.params, T, B), B, output.params)
            aicc = aic + 2 * 4 * ((4 + 1)/(len(T) - 4 - 1))

            if aic < results_all["sch.aic"] or results_all["sch.aic"] == [np.NaN] and rsq >= -1:
                best_p = output.params
                results_all = {
                    "sch.id": [id],
                    "sch.B0": [output.params["B0"].value],
                    "sch.E": [output.params["E"].value],
                    "sch.Th": [output.params["Th"].value],
                    "sch.Eh": [output.params["Eh"].value],
                    "sch.chisqr": [output.chisqr],
                    "sch.Rsq": rsq,
                    "sch.aic": aic,
                    "sch.bic": bic,
                    "sch.aicc":aicc,
                    "sch.Rsq_log": rsq_log,
                    "sch.aic_log": bic_log,
                    "sch.bic_log": aic_log,
                    "sch.aicc_log": aicc_log
                }
        except ValueError:
            pass
        continue

    if only_return_info == True:
        return (pd.DataFrame(results_all)), ArrInf_bef, ArrInfo_af
    else:
        return (pd.DataFrame(results_all)), best_p, ArrInf_bef, ArrInfo_af

### (3) Define Arrhenius Model

In [8]:
def Arrhenius_model(id, data):
    # k = 8.617e-5
    sub = data[data["ID"] == id]
    T = np.asarray(data["ConTemp"][data['ID'] == id])  # Temperature in kelvin
    # B_log = np.asarray(data["LoggedOriginalTraitValue"][data['ID'] == id])  # Unlogged Trait value
    
    T_max = sub["Transferred_ConTemp"][sub["LoggedOriginalTraitValue"] == np.max(sub["LoggedOriginalTraitValue"])]

    if len(T_max) >= 2:
        T_max = np.min(T_max)

    # B_log_max = sub["LoggedOriginalTraitValue"][sub["Transferred_ConTemp"] == np.min(
    #     sub["Transferred_ConTemp"])]
    # if len(B_log_max) >= 2:
    #     T_max = np.mean(B_log_max)

    B_log = np.asarray(sub["LoggedOriginalTraitValue"])
    Trans_T = np.asarray(sub["Transferred_ConTemp"])
    logB_bef = np.asarray(sub["LoggedOriginalTraitValue"][sub["Transferred_ConTemp"] <= float(T_max)])
    logB_af = np.asarray(sub["LoggedOriginalTraitValue"][sub["Transferred_ConTemp"] >= float(T_max)])
    transT_bef = np.asarray(
        sub["Transferred_ConTemp"][sub["Transferred_ConTemp"] <= float(T_max)])  # Transferred Temperature
    transT_af = np.asarray(
        sub["Transferred_ConTemp"][sub["Transferred_ConTemp"] >= float(T_max)])
    
    fit_arr = np.polyfit(Trans_T, B_log, 1)
    B0_all = fit_arr[0]

    # Get fitting info of Arrhenius model of dataset BEFORE deactivation
    if len(logB_bef) > 1:
        fitArr_bef = np.polyfit(transT_bef, logB_bef, 1)
        resi_bef = resi(fitArr_bef, transT_bef, logB_bef)

        rss_bef = sum(resi_bef ** 2)
        tss_bef = sum((logB_bef - np.mean(logB_bef)) ** 2)
        rsq_bef = 1 - (rss_bef/tss_bef)

        aic_bef = get_AIC(resi_bef, logB_bef, fitArr_bef)
        bic_bef = get_BIC(resi_bef, logB_bef, fitArr_bef)
        #aicc = aic + (2 * k * (k + 1)) / (len(T) - k - 1)
        aicc_bef = aic_bef + (2 * 2 * (2 + 1)) / ((len(T) - 2 - 1))

        results_all_before = {
            "id": [id],
            "B0_log_before_deactivation": fitArr_bef[0],
            "E_before_deactivation": fitArr_bef[1],
            "Rsq_before_deactivation": rsq_bef,
            "aic_before_deactivation": aic_bef,
            "bic_before_deactivation": bic_bef,
            "aicc_before_deactivation": aicc_bef
        }

    else:
        results_all_before = {
            "id": [id],
            "B0_log_before_deactivation": B0_all,
            "E_before_deactivation": 0.5,
            "Rsq_before_deactivation": np.NaN,
            "aic_before_deactivation": np.NaN,
            "bic_before_deactivation": np.NaN,
            "aicc_before_deactivation": np.NaN
        }

    # Get fitting info of Arrhenius model of dataset AFTER deactivation
    if len(logB_af) > 1:
        fitArr_af = np.polyfit(transT_af, logB_af, 1)
        resi_af = resi(fitArr_af, transT_af, logB_af)

        rss_af = sum(resi_af ** 2)
        tss_af = sum((logB_af - np.mean(logB_af)) ** 2)
        rsq_af = 1 - (rss_af/tss_af)

        aic_af = get_AIC(resi_af, logB_af, fitArr_af)
        bic_af = get_BIC(resi_af, logB_af, fitArr_af)
        aicc_af = aic_af + 2 * 2 * ((2 + 1)/(len(T) - 2 - 1))

        results_all_after = {
            "id": [id],
            "B0_log_after_deactivation": fitArr_af[0],
            "E_after_deactivation": fitArr_af[1],
            "Rsq_after_deactivation": rsq_af,
            "aic_after_deactivation": aic_af,
            "bic_after_deactivation": bic_af,
            "aicc_after_deactivation": aicc_af
        }
    else:
        results_all_after = {
            "id": [id],
            "B0_log_after_deactivation": B0_all,
            "E_after_deactivation": 0.5,
            "Rsq_after_deactivation": np.NaN,
            "aic_after_deactivation": np.NaN,
            "bic_after_deactivation": np.NaN,
            "aicc_after_deactivation": np.NaN
        }
    return pd.DataFrame(results_all_before), pd.DataFrame(results_all_after)


### (4) Define Linear Modelling Function

In [9]:
def linear_models(id, data, only_return_info = True):
    ''' Fitting three linear model using OLS method '''
    T = np.asarray(data["ConTemp"][data['ID'] == id])  # Temperature in kelvin
    B = np.asarray(data["OriginalTraitValue"]
                   [data['ID'] == id])  # Unlogged Trait value

    # Get fitting info of quadratic model
    fit_quad = np.polyfit(T, B, 2)
    resi2 = resi(fit_quad, T, B)
    rss2 = sum(resi2 ** 2)
    tss2 = sum((B - np.mean(B)) ** 2)
    rsq2 = 1 - (rss2/tss2)
    aic2 = get_AIC(resi2, B, fit_quad)
    bic2 = get_BIC(resi2, B, fit_quad)
    aicc2 = aic2 + 2 * 3 * ((3 + 1)/(len(T) - 3 - 1))

    results_all_quad = {
        "quad.id": [id],
        "quad.a": fit_quad[0],
        "quad.b1": fit_quad[1],
        "quad.b2": fit_quad[2],
        "quad.Rsq": rsq2,
        "quad.aic": aic2,
        "quad.bic": bic2,
        "quad.aicc": aicc2
    }

    # Get fitting info of cubic model
    fit_cubic = np.polyfit(T, B, 3)
    resi3 = resi(fit_cubic, T, B)
    rss3 = sum(resi3 ** 2)
    tss3 = sum((B - np.mean(B)) ** 2)
    rsq3 = 1 - (rss3/tss3)
    aic3 = get_AIC(resi3, B, fit_cubic)
    bic3 = get_BIC(resi3, B, fit_cubic)
    aicc3 = aic3 + 2 * 4 * ((4 + 1)/(len(T) - 4 - 1))

    results_all_cubic = {
        "cubic.id": [id],
        "cubic.a": fit_cubic[0],
        "cubic.b1": fit_cubic[1],
        "cubic.b2": fit_cubic[2],
        "cubic.b3": fit_cubic[3],
        "cubic.Rsq": rsq3,
        "cubic.aic": aic3,
        "cubic.bic": bic3,
        "cubic.aicc": aicc3
    }

    if only_return_info == True:
        return pd.DataFrame(results_all_quad), pd.DataFrame(results_all_cubic)
    else:
        return pd.DataFrame(results_all_quad), pd.DataFrame(results_all_cubic), fit_quad, fit_cubic

## Main Program

In [10]:
def model_fitting(data):
    Arrhenius_after = pd.DataFrame(data=None)
    Arrhenius_before = pd.DataFrame(data=None)
    quadratic = pd.DataFrame(data=None)
    cubic = pd.DataFrame(data=None)
    Briere = pd.DataFrame(data=None)
    Schoolfield = pd.DataFrame(data=None)
    data_name = data["StandardisedTraitName"].unique()[0].replace(" ", "")

    ## Model Fitting for Net Phototh data
    for id in data["ID"].unique():
        if len(data[data["ID"] == id]) > 5:
            print("Fitting Models of ID %d" % id)
            quad_info, cubic_info, fit_quad, fit_cubic = linear_models(id, data, only_return_info=False)
            quadratic = quadratic.append(quad_info)
            cubic = cubic.append(cubic_info)

            # print("Fitting Briere Modle of ID %d" % id)
            briere_info, p_bri = Briere_model(id, data, min_round=230, max_round=280, only_return_info=False)
            Briere = Briere.append(briere_info)

            # print("Fitting Simplified Schoolfield Modle of ID %d" % id)
            school_info, p_sch, arr_before, arr_after = Schoolfield_model(id, data, min_round=280, max_round=320, only_return_info=False)
            Schoolfield = Schoolfield.append(school_info)
            Arrhenius_before = Arrhenius_before.append(arr_before)
            Arrhenius_after = Arrhenius_after.append(arr_after)
            
            # Save pictures
            models_visualisation(id, data, (fit_quad, fit_cubic, p_bri, p_sch))
            try:
                plt.savefig("../../Results/%s/Figures/TPC_fitting%d.pdf" % (data_name,id))
            except FileNotFoundError:
                print(os.system("mkdir -p ../../Results/"+data_name+"/Figures"))
                plt.savefig("../../Results/%s/Figures/TPC_fitting%d.pdf" % (data_name,id))
            plt.close()
        else:
            print("ID %d has only %d observation(s)" % (id, len(d[d["ID"] == id])))

    print("-"*80, '\n')
    try:
        quadratic.to_csv("../../Results/"+data_name+"/fitInfos/quad_info.csv", sep=",", index=False)
        cubic.to_csv("../../Results/"+data_name+"/fitInfos/cubic_info.csv", sep=",", index=False)
        Briere.to_csv("../../Results/"+data_name+"/fitInfos/Briere_info.csv", sep=",", index=False)
        Schoolfield.to_csv("../../Results/"+data_name+"/fitInfos/Schoolfield_info.csv", sep=",", index=False)
        Arrhenius_before.to_csv("../../Results/"+data_name+"/fitInfos/Arrhenius_before_info.csv")
        Arrhenius_after.to_csv("../../Results/"+data_name+"/fitInfos/Arrhenius_after_info.csv")

    except FileNotFoundError:
        print(os.system("mkdir ../../Results/"+data_name+"/fitInfos"))
        quadratic.to_csv("../../Results/"+data_name+"/fitInfos/quad_info.csv", sep=",", index=False)
        cubic.to_csv("../../Results/"+data_name+"/fitInfos/cubic_info.csv", sep=",", index=False)
        Briere.to_csv("../../Results/"+data_name+"/fitInfos/Briere_info.csv", sep=",", index=False)
        Schoolfield.to_csv("../../Results/"+data_name+"/fitInfos/Schoolfield_info.csv", sep=",", index=False)
        Arrhenius_before.to_csv("../../Results/"+data_name+"/fitInfos/Arrhenius_before_info.csv")
        Arrhenius_after.to_csv("../../Results/"+data_name+"/fitInfos/Arrhenius_after_info.csv")

    return "Model Fitting for "+ data_name +" Rate Finished!"

In [79]:
k = 8.617e-5
d = pd.DataFrame(pd.read_csv("../../Data/ThermRespData.csv"))
dd = d.groupby(['ID']).agg(['count']);
d = d[d['OriginalTraitValue'] > 0]
d['LoggedOriginalTraitValue'] = np.log(d['OriginalTraitValue'])
d = d[["ID", "ConTemp", "OriginalTraitValue", "StandardisedTraitName",
    "LoggedOriginalTraitValue", "OrignalTraitName",  "OriginalTraitUnit"]]
d["ConTemp_Kelvin"] = d["ConTemp"] + 273.15
d["Transferred_ConTemp"] = 1/k*d['ConTemp_Kelvin']+1/283.15*k
d["Transferred_kT"] = 1/k*d["ConTemp_Kelvin"]

# Find the subset that has different biological indicators
d_photosyn = d.loc[(d['StandardisedTraitName'] == d['StandardisedTraitName'].unique()[0]) | (d['StandardisedTraitName'] == d['StandardisedTraitName'].unique()[1])]
d_respiration = d.loc[d['StandardisedTraitName'] == d['StandardisedTraitName'].unique()[2]]

In [130]:
def find_curve(d, bef_thre, af_thre, all = 5):
    df = pd.DataFrame(data=None)
    for i in d["ID"].unique():
        sub = d.loc[d["ID"] == i]
        sub_agg = sub.groupby(["ID"]).agg(["count"])[["ConTemp"]]
        T = np.asarray(sub["ConTemp"])  # Temperature in kelvin   
        B = np.asarray(sub["OriginalTraitValue"])
        T_max = sub["ConTemp"][sub["OriginalTraitValue"] == np.max(sub["OriginalTraitValue"])]
        T_min = sub["ConTemp"][sub["OriginalTraitValue"] == np.min(sub["OriginalTraitValue"])]

        if len(T_max) >= 2:
            T_max = np.mean(T_max)

        if len(T_min) >= 2:
            T_min = np.mean(T_min)

        # bef_rows = sub[sub["ConTemp"] <= float(T_max)].shape[0]
        # af_rows = sub[sub["ConTemp"] >= float(T_max)].shape[0]; #print(bef_rows, af_rows, sub.shape[0])

        bef_rows = sub[sub["ConTemp"] <= float(T_min)].shape[0]
        af_rows = sub[sub["ConTemp"] >= float(T_min)].shape[0]; #print(bef_rows, af_rows, sub.shape[0])

        data = {"ID": [i],
                "Count": [sub.shape[0]],
                "Count_before": [bef_rows],
                "count_after": [af_rows]}

        df2 = pd.DataFrame(data = data) #df2

        if bef_rows >= bef_thre and af_rows >= af_thre and sub.shape[0] > all:
            df = df.append(df2)

    return df

In [133]:
find_curve(d_photosyn, 5, 5, 5)

,ID,Count,Count_before,count_after
0,289,14,5,9
0,312,28,6,23
0,318,359,30,330
0,519,9,5,5


In [134]:
find_curve(d_respiration, 3, 3, 5)

,ID,Count,Count_before,count_after
0,46,8,3,6
0,108,6,3,4
0,237,46,6,43
0,254,12,9,4
0,255,7,5,3
0,261,8,3,6
0,263,8,3,6
0,273,10,8,3
0,303,23,4,20
0,306,6,3,4


In [112]:
find_curve(d_photosyn, 2, 3, inverse = True)

,ID,Count,Count_before,count_after
0,9,11,2,10
0,23,7,2,6
0,42,6,2,5
0,79,9,1,9
0,142,16,2,15
0,266,6,2,5
0,271,10,2,9
0,297,9,1,9
0,299,7,1,7
0,365,7,2,6


In [54]:
''' FIND THE NUMBER OF ROWS FOR EACH SUBSET GROUP BY ID '''

photosyn_id = d_photosyn.groupby(['ID']).agg(['count']); 
print(photosyn_id.loc[photosyn_id.iloc[:,1] > 5].shape[0])
respiration_id = d_respiration.groupby(['ID']).agg(['count']); 
print(respiration_id.loc[respiration_id.iloc[:,1] > 5].shape[0])

378
249


In [ ]:
model_fitting(d_photosyn)

In [ ]:
model_fitting(d_respiration)